In [78]:
import os
import json
from glob import glob
import urllib
import requests

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

In [95]:
def load(name: str='magazine', root_dir: str='../raw/'):
    PATH = {
        'magazine': os.path.join(root_dir, 'magazine.json'), 
        'metadata': os.path.join(root_dir, 'metadata.json'), 
        'users': os.path.join(root_dir, 'users.json'),
        'dev': os.path.join(root_dir, 'dev.users'),
        'test': os.path.join(root_dir, 'test.users')
        }
    
    if name in ['magazine', 'metadata', 'users']:
        data = pd.DataFrame([json.loads(line) for line in open(PATH[name], 'r', encoding='utf-8')])
    elif name in ['dev', 'test']:
        data = pd.read_csv(PATH['dev'], header=None)
    else:
        raise NotImplementedError()

    return data

In [96]:
magazine = load(name='magazine')
metadata = load(name='metadata')
users = load(name='users')

In [97]:
users

,keyword_list,following_list,id
0,[],"[@perytail, @brunch]",#901985d8bc4c481805c4a4f911814c4a
1,[],"[@holidaymemories, @wadiz, @sciforus, @dailydu...",#1fd89e9dcfa64b45020d9eaca54e0eed
2,[],"[@commerceguy, @sunsutu, @kakao-it, @joohoonja...",#1d94baaea71a831e1f33e1c6bd126ed5
3,[],"[@amberjeon48, @forsy20, @nemotokki, @hawann, ...",#04641c01892b12dc018b1410e4928c0d
4,[],"[@dwcha7342, @iammento, @kakao-it, @dkam, @ant...",#65bcaff862aadff877e461f54187ab62
...,...,...,...
310753,[],"[@login002, @kkonal, @leeraha, @tobeme, @sohyu...",#2863e47d50f1640df6dac10b7bad94fb
310754,[],"[@simplelife-1p, @mint5051, @thecapitalist, @s...",#4341a155d1966e5618e310c45386aea4
310755,[],"[@cometseeker, @bijou, @suhanjang, @brunch]",#0d70f397a78d2ef638f812592fa8e6ba
310756,"[{'cnt': 1, 'keyword': '꽃병 꽃꽂이'}, {'cnt': 1, '...","[@taekangk, @cielbleu, @yongisa, @joongheekim,...",#1bbf5e3d1e4c373103981cdd819812da
